In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
from collections import deque
import yfinance as yf
import pandas as pd

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
def get_stock_data(path):
    df = pd.read_csv(path)
    df = df[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()
    return df

In [3]:
from hmmlearn import hmm
from backtesting import Backtest, Strategy

# Load historical stock data
df = get_stock_data("/home/artzuros/Documents/CS/Trader/trading/data/INDEX/NIFTY 50.csv")

In [4]:
df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1)).dropna()
X = df[['LogReturn']].dropna().values 

In [5]:
n_states = 3  # Number of hidden states (regimes)
model = hmm.GaussianHMM(n_components=n_states, covariance_type="full", n_iter=100)
model.fit(X)

# Predict hidden states (market regimes) based on the log returns
hidden_states = model.predict(X)
df['State'] = np.concatenate(([np.nan], hidden_states))  # Add NaN for the first row

# Display the inferred hidden states
print("Inferred Hidden States:")
print(df[['Close', 'State']].head(10))

Inferred Hidden States:
       Close  State
1413  939.55    NaN
1414  927.05    0.0
1415  931.65    0.0
1416  927.80    0.0
1417  928.80    0.0
1418  946.70    0.0
1419  956.60    0.0
1420  980.35    0.0
1421  980.70    0.0
1422  989.30    0.0


In [8]:
df.tail()

,Open,High,Low,Close,Volume,LogReturn,State
7505,15839.35,15870.80,15772.30,15860.35,314614380.0,0.004417,1.0
7506,15915.35,15915.65,15792.15,15814.70,255099272.0,-0.002882,1.0
7507,15807.50,15835.90,15724.05,15748.45,360334724.0,-0.004198,1.0
7508,15776.90,15839.10,15708.75,15721.50,262386323.0,-0.001713,1.0
7509,15755.05,15755.55,15667.05,15680.00,224921644.0,-0.002643,1.0


In [ ]:
# Define a backtest strategy based on HMM states
class HMMStrategy(Strategy):
    def init(self):
        # Preload the states from the DataFrame
        self.states = self.data.df['State'].values

    def next(self):
        # Get the current bar index
        bar_index = len(self.data) - 1
        
        # Get the hidden state corresponding to the current bar index
        current_state = self.states[bar_index]
        
        # Trading logic based on the inferred state
        if current_state == 1 and not self.position:  # Bullish state - Buy if no position
            self.buy()
        elif current_state == 2 and self.position:  # Bearish state - Sell if in position
            self.sell()
        # State 0: Hold in a neutral state

test_df =df[['Open', 'High', 'Low', 'Close', 'Volume']]

# Run backtest with the modified HMM strategy
bt = Backtest(df, HMMStrategy, cash=100000, commission=0.002)
stats = bt.run()
print(stats)
bt.plot()

/home/artzuros/miniconda3/envs/trader/lib/python3.10/site-packages/backtesting/backtesting.py:1095: FutureWarning: Index.is_numeric is deprecated. Use pandas.api.types.is_any_real_numeric_dtype instead
  (data.index.is_numeric() and
/tmp/ipykernel_15641/506731803.py:22: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, HMMStrategy, cash=100000, commission=0.002)


Start                                  1413.0
End                                    7509.0
Duration                               6096.0
Exposure Time [%]                   98.638675
Equity Final [$]                 1481785.3172
Equity Peak [$]                  1492520.1172
Return [%]                        1381.785317
Buy & Hold Return [%]              1568.88404
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -59.773935
Avg. Drawdown [%]                   -3.892167
Max. Drawdown Duration                  966.0
Avg. Drawdown Duration              33.730994
# Trades                                  1.0
Win Rate [%]                            100.0
Best Trade [%]                    1393.077846
Worst Trade [%]                   1393.077846
Avg. Trade [%]                    

GridPlot(id='p1294', ...)

Opening in existing browser session.
